Data Loading and Preprocessing

In [ ]:
import pandas as pd
# Load data
data = pd.read_csv(r'C:\Users\cex\Desktop\Recommender Systems Practical Study\Us-Ecommerce Dataset.csv')

# Display basic info
print(data.info())
print(data.head())

# Handling missing values
data.fillna(method='ffill', inplace=True)  # Simple forward fill for missing values

# Convert columns to appropriate types if needed
data['Date'] = pd.to_datetime(data['Date'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56082 entries, 0 to 56081
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Transaction_id        10999 non-null  float64
 1   customer_id           10999 non-null  float64
 2   Date                  10999 non-null  object 
 3   Product               10999 non-null  object 
 4   Gender                10999 non-null  object 
 5   Device_Type           10999 non-null  object 
 6   Country               10999 non-null  object 
 7   State                 10999 non-null  object 
 8   City                  10999 non-null  object 
 9   Category              10999 non-null  object 
 10  Customer_Login_type   10999 non-null  object 
 11  Delivery_Type         10999 non-null  object 
 12   Quantity             10999 non-null  float64
 13  Transaction Start     10999 non-null  float64
 14  Transaction_Result    10999 non-null  float64
 15  Amount US$         

C:\Users\cex\AppData\Local\Temp\ipykernel_23188\2604989418.py:3: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r'C:\Users\cex\Desktop\Recommender Systems Practical Study\Us-Ecommerce Dataset.csv')
C:\Users\cex\AppData\Local\Temp\ipykernel_23188\2604989418.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)  # Simple forward fill for missing values
C:\Users\cex\AppData\Local\Temp\ipykernel_23188\2604989418.py:13: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Date'] = pd.to_datetime(data['Date'])


Data Splitting

In [43]:
%pip install scikit-learn
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Algorithm Implementation

User-Based Collaborative Filtering

In [28]:
# Remove any leading or trailing whitespace from column names
train_data.columns = train_data.columns.str.strip()

In [29]:
# Create a pivot table after cleaning column names
user_product_matrix = train_data.pivot_table(index='customer_id', columns='Product', values='Quantity', fill_value=0)

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

# Create a pivot table
user_product_matrix = train_data.pivot_table(index='customer_id', columns='Product', values='Quantity', fill_value=0)

# Compute similarity between users
user_similarity = cosine_similarity(user_product_matrix)


 Item-Based Collaborative Filtering

In [31]:
# Transpose the matrix for item-based filtering
item_product_matrix = user_product_matrix.T
item_similarity = cosine_similarity(item_product_matrix)

# Convert similarity into a DataFrame
item_similarity_df = pd.DataFrame(item_similarity, index=item_product_matrix.index, columns=item_product_matrix.index)

Matrix Factorization with SVD

In [32]:
from scipy.sparse import csr_matrix
# Convert user_product_matrix to a sparse matrix format
user_product_sparse_matrix = csr_matrix(user_product_matrix.values)

In [33]:
# Check the shape of the matrix
num_rows, num_cols = user_product_sparse_matrix.shape
print(f"Matrix shape: {num_rows} rows, {num_cols} columns")

Matrix shape: 8746 rows, 3 columns


In [34]:
from scipy.sparse.linalg import svds
import numpy as np

# Set k to 2, as it's the maximum allowable value for this matrix shape
k = 2

# Perform SVD with the adjusted k
U, sigma, Vt = svds(user_product_sparse_matrix, k=k)

# Convert sigma to a diagonal matrix
sigma = np.diag(sigma)

Content-Based Filtering

In [35]:
# Example: Generate item profiles based on features like 'Category' or 'Device_Type'
item_profiles = pd.get_dummies(data[['Product', 'Category', 'Device_Type']])

Evaluation

Evaluation Metrics

In [36]:
print(test_data.columns)

Index(['Transaction_id', 'customer_id', 'Date', 'Product', 'Gender',
       'Device_Type', 'Country', 'State', 'City', 'Category',
       'Customer_Login_type', 'Delivery_Type', ' Quantity ',
       'Transaction Start', 'Transaction_Result', 'Amount US$',
       'Individual_Price_US$', 'Year_Month', 'Time'],
      dtype='object')


In [37]:
test_data.columns = test_data.columns.str.strip()

In [38]:
print(test_data.columns)

Index(['Transaction_id', 'customer_id', 'Date', 'Product', 'Gender',
       'Device_Type', 'Country', 'State', 'City', 'Category',
       'Customer_Login_type', 'Delivery_Type', 'Quantity', 'Transaction Start',
       'Transaction_Result', 'Amount US$', 'Individual_Price_US$',
       'Year_Month', 'Time'],
      dtype='object')


In [39]:
# Reconstruct the predicted user-product matrix
predicted_matrix = np.dot(np.dot(U, sigma), Vt)

# Convert it to a DataFrame for easier indexing
predicted_ratings_df = pd.DataFrame(predicted_matrix, index=user_product_matrix.index, columns=user_product_matrix.columns)

In [40]:
# Ensure that test_data has customer_id and Product columns
predictions = []
for _, row in test_data.iterrows():
    customer_id = row['customer_id']
    product = row['Product']
    # Check if the prediction exists in the matrix
    if customer_id in predicted_ratings_df.index and product in predicted_ratings_df.columns:
        predicted_quantity = predicted_ratings_df.loc[customer_id, product]
    else:
        # Use a default prediction if no data is available
        predicted_quantity = 0
    predictions.append(predicted_quantity)

# Convert predictions to a Series to match test_data format
predictions = pd.Series(predictions)

In [41]:
from sklearn.metrics import mean_squared_error

# Calculate Mean Squared Error between actual and predicted quantities
mse = mean_squared_error(test_data['Quantity'], predictions)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 61.59472229651422


In [42]:
import pickle
model = dtc
pickle.dump(model, open("E-commerce_Recommender_System.pkl", 'wb'))